# Ceneo Scraper

## Import bibliotek

In [ ]:
import requests
from bs4 import BeautifulSoup

## Strona produktu

In [ ]:
product_id = '84514582'
url = f'https://www.ceneo.pl/{product_id}#tab=reviews'

## Wysłanie żądania dostępu do zasobu

In [ ]:
response = requests.get(url)
response.status_code

In [ ]:
page = BeautifulSoup(response.text, 'html.parser')
opinions = page.select('.js_product-review')
opinion = page.select_one('.js_product-review')